# Opening bottle with sparkling water

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Mar 31th, 2022</p>

```{attention}
Always make sure you are using the [latest version of Reaktoro](https://anaconda.org/conda-forge/reaktoro). Otherwise, some new features documented on this website will not work on your machine and you may receive unintuitive errors. Follow these [update instructions](updating_reaktoro_via_conda) to get the latest version of Reaktoro!
```

This tutorial shows how to simulate the solubility of CO<sub>2</sub> in water or, more simply, the **effect of the carbon dioxide released when you open the bottle of sparkling water**.

|![Opening bottle with sparkling water](../../images/applications/opening-bottle-with-soda.jpg)|
|:--:|
|Opening bottle with sparkling water, Source: flavorman.com|

First, we define the chemical system:

In [1]:
from reaktoro import *

db = SupcrtDatabase("supcrtbl")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("H O C"))
aqueousphase.set(ActivityModelPitzer())

# Create a gaseous phase
gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.set(ActivityModelPengRobinson())

# Create the chemical system
system = ChemicalSystem(db, aqueousphase, gaseousphase)

# Create the equilibrium solver
solver = EquilibriumSolver(system)

Then, we define the range of pressures using the `linspace()` function of the **numpy** library. The initial and final pressures correspond to the values in the bubble bottle before and after opening.

In [2]:
import numpy as np
closedP = 3.79 # in bars
openP = 1.01325 # in bars
pressures = np.linspace(openP, closedP, num=100)

> **Note**: A typical carbonated soft drink contains approximately 3–4 volumes (6–8 g/L) CO<sub>2</sub>. To obtain the amount of mol of CO<sub>2</sub>, we need to perform the following calculations: 8 g/L = 8 / 44.01 mol = 0.18 mol, where 44.01 g/mol is the CO<sub>2</sub> molar mass.

Next, we go through the created pressure list and collect the CO<sub>2</sub>(g) amounts obtained in the equilibrated chemical for a given pressure.

In [3]:
import pandas as pd
df = pd.DataFrame(columns=["P", "amountCO2"])

for P in pressures:
    state = ChemicalState(system)
    state.setTemperature(20.0, "celsius")
    state.setPressure(P, "bar")
    state.add("H2O(aq)", 0.5, "kg")   # add ~ half a liter of water
    state.add("CO2(g)", 0.18, "mol") # add calculated amount of gas

    res = solver.solve(state)

    df.loc[len(df)] = [P, float(state.speciesAmount("CO2(g)"))]

To visualize the changes in the CO<sub>2</sub>(g) amount in the bottle, we export [bokeh](https://docs.bokeh.org/en/latest/docs/gallery.html#standalone-examples) python plotting package.

In [4]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

hovertool = HoverTool()
hovertool.tooltips = [("amount(CO2) in brine", "@amountCO2 mol"), 
                      ("P", "@P")]

p = figure(
    title="CO2(G) AMOUNT IN SPARKLING WATER BOTTLE BEFORE AND AFTER OPENING",
    x_axis_label=r'PRESSURE [BAR]',
    y_axis_label='AMOUNT OF CO2(G) [MOL]',
    sizing_mode="scale_width",
    height=300)

p.add_tools(hovertool)

p.line("P", "amountCO2", line_width=3, line_cap="round", line_color='indigo', source=df)

show(p)

Loading BokehJS ...

From the generated plot, it can be seen that reducing the pressure in the bottle (opening the bottle) also reduces the amount of CO<sub>2</sub> dissolved in the carbonated beverage, which evaporates as CO<sub>2</sub>  gas.